# Librerías

In [38]:
import pandas as pd
import os


import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# Delitos

In [4]:
# Lectura de datasets de delitos

# Directorio donde se encuentran los archivos CSV
directorio = '../Datasets/Raw'

# Obtener la lista de archivos CSV en el directorio
archivos_csv = [archivo for archivo in os.listdir(directorio) if archivo.startswith('delitos_') and archivo.endswith('.csv')]

# Inicializar un DataFrame vacío para almacenar los datos
delitos = pd.DataFrame()

# Leer cada archivo CSV y concatenarlos en el DataFrame final
for archivo in archivos_csv:
    ruta_archivo = os.path.join(directorio, archivo)
    df_temporal = pd.read_csv(ruta_archivo)
    delitos = pd.concat([delitos, df_temporal], ignore_index=True)



# Lectura de otros datasets
comisarias = pd.read_csv('../Datasets/comisarias-policia-de-la-ciudad.csv')

from urllib.request import urlopen
import json
with urlopen('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-educacion/barrios/barrios.geojson') as response:
    barrios_geojson = json.load(response)



# Modificar tipo de dato
delitos['fecha'] = pd.to_datetime(delitos['fecha'])
delitos['franja'] = delitos['franja'].astype('Int32')
delitos['comuna'] = delitos['comuna'].astype('Int32')


# Añadir nuevas columnas
delitos['mes_numero'] = delitos['fecha'].dt.month
delitos['dia_numero'] = delitos['fecha'].dt.isocalendar().day
delitos['dia_mes_numero'] = delitos['fecha'].dt.day
#delitos['anio_mes'] = delitos['fecha'].dt.to_period('M')
delitos['anio_mes'] = pd.to_datetime(delitos['fecha'], format='%Y-%m').dt.to_period('M')


# Eliminamos registros que no nos interesan para el análisis


delitos = delitos[-delitos['tipo'].isin(['Homicidios', 'Amenazas', 'Lesiones'])]


# Seleccionamos solo registros de robos y hurtos

robos = delitos[delitos['subtipo'].isin(['Hurto total', 'Robo total'])]


# Eliminamos los registros que no tienen ubicación

cols = ['barrio', 'comuna', 'latitud', 'longitud']
robos = robos.dropna(subset = cols).reset_index(drop=True)

# Guardamos el fichero de robos en un dataframe
#robos.to_csv('../Datasets/robos_all_years.csv', index=False)


In [6]:
robos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653525 entries, 0 to 653524
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id-mapa         653525 non-null  int64         
 1   anio            653525 non-null  int64         
 2   mes             653525 non-null  object        
 3   dia             653525 non-null  object        
 4   fecha           653525 non-null  datetime64[ns]
 5   franja          651603 non-null  Int32         
 6   tipo            653525 non-null  object        
 7   subtipo         653525 non-null  object        
 8   uso_arma        653525 non-null  object        
 9   uso_moto        653525 non-null  object        
 10  barrio          653525 non-null  object        
 11  comuna          653525 non-null  Int32         
 12  latitud         653525 non-null  float64       
 13  longitud        653525 non-null  float64       
 14  cantidad        653525 non-null  int

In [23]:
#robos[robos['franja'].isna()]
robos.franja.value_counts()

franja
20    41061
19    40402
18    39531
21    35695
17    35120
12    34043
14    33330
16    33272
15    32626
0     32435
13    31736
22    31331
10    28679
11    28143
8     27010
9     26493
23    25282
7     22027
6     16612
1     13218
5     13042
2     10829
4      9886
3      9800
Name: count, dtype: Int64

# Weather

In [29]:
weather = pd.read_csv('../Datasets/Raw/weather_data.csv')
weather

,time,w_temperatura,w_humedad_relativa,w_precipitaciones,w_velocidad_del_viento,w_presion_del_aire,w_codigo_del_tiempo
0,2017-01-01 00:00:00,24.7,88.0,NaN,5.4,1005.0,NaN
1,2017-01-01 01:00:00,22.8,91.0,NaN,7.2,1005.6,NaN
2,2017-01-01 02:00:00,22.6,92.0,NaN,4.3,1005.5,NaN
3,2017-01-01 03:00:00,22.5,96.0,NaN,3.2,1004.7,NaN
4,2017-01-01 04:00:00,21.3,95.0,NaN,2.2,1005.6,NaN
...,...,...,...,...,...,...,...
52579,2022-12-31 19:00:00,24.5,58.0,0.0,17.6,1013.8,1.0
52580,2022-12-31 20:00:00,23.6,61.0,0.0,17.3,1013.8,1.0
52581,2022-12-31 21:00:00,22.4,73.0,0.0,16.6,1013.7,1.0
52582,2022-12-31 22:00:00,21.2,80.0,0.0,16.2,1013.8,1.0


In [35]:
weather['date'] = weather['time'].str.split(' ', expand=True)[0]
weather['hour'] = weather['time'].str.split(' ', expand=True)[1].str.split(':', expand=True)[0]
weather['hour'] = weather['hour'].astype('Int64')

weather.to_csv('../Datasets/Clean/weather_data_clean.csv', index = False)

In [34]:
weather

,time,w_temperatura,w_humedad_relativa,w_precipitaciones,w_velocidad_del_viento,w_presion_del_aire,w_codigo_del_tiempo,date,hour
0,2017-01-01 00:00:00,24.7,88.0,NaN,5.4,1005.0,NaN,2017-01-01,0
1,2017-01-01 01:00:00,22.8,91.0,NaN,7.2,1005.6,NaN,2017-01-01,1
2,2017-01-01 02:00:00,22.6,92.0,NaN,4.3,1005.5,NaN,2017-01-01,2
3,2017-01-01 03:00:00,22.5,96.0,NaN,3.2,1004.7,NaN,2017-01-01,3
4,2017-01-01 04:00:00,21.3,95.0,NaN,2.2,1005.6,NaN,2017-01-01,4
...,...,...,...,...,...,...,...,...,...
52579,2022-12-31 19:00:00,24.5,58.0,0.0,17.6,1013.8,1.0,2022-12-31,19
52580,2022-12-31 20:00:00,23.6,61.0,0.0,17.3,1013.8,1.0,2022-12-31,20
52581,2022-12-31 21:00:00,22.4,73.0,0.0,16.6,1013.7,1.0,2022-12-31,21
52582,2022-12-31 22:00:00,21.2,80.0,0.0,16.2,1013.8,1.0,2022-12-31,22


# Semáforos

In [36]:
semaforos = pd.read_csv('../Datasets/Raw/cruces-semaforizados.csv')

In [39]:
semaforos

,long,lat,n_cruce,id_codigo,fecha_inau,anio_inau,dom_orig,dom_norma,origen,observ,nro_nva,instala,horario,fecha_fina,id_usig,objeto,calle_nombre,calle_altura,barrio,comuna,codigo_postal,codigo_postal_argentino
0,-58.369318,-34.609241,1015.0,B-CR-01015,NaN,NaN,Paseo Colón Av. y La Rábida,NaN,Coordenada,Av. Paseo Colón - Nueva Rábida (Cruce nuevo a ...,4.0,TITILANTE A PEDIDO DE GCBA,Titilante por obra,Hasta la finalización de Obra de Paseo del Baj...,1,CRUCE SEMAFORIZADO,PASEO COLON AV.,NaN,Monserrat,Comuna 1,NaN,NaN
1,-58.367144,-34.612145,1031.0,B-CR-01031,NaN,NaN,Belgrano Av. y Huergo Ing. Av.,NaN,Coordenada,Huergo entre Av. Belgrano y Venezuela (provis...,4.0,APAGADO A PEDIDO DE GCBA,Apagado por obra,Hasta la finalización de Obra de Paseo del Baj...,2,CRUCE SEMAFORIZADO,BELGRANO AV.,NaN,Monserrat,Comuna 1,NaN,NaN
2,-58.402424,-34.589907,1145.0,B-CR-01145,NaN,NaN,Laprida 1739,NaN,Coordenada,Laprida -Salida de Bomberos,4.0,TITILANTE CON DEMANDA,Titilante 24hs.,NaN,3,CRUCE SEMAFORIZADO,LAPRIDA,1739.0,Recoleta,Comuna 2,1425.0,C1425EKO
3,-58.381459,-34.623798,2132.0,B-CR-02132,NaN,NaN,Lima y Cochabamba,NaN,Coordenada,Lima-Cochabamba,4.0,TITILANTE A PEDIDO DE GCBA,Titilante 24hs.,NaN,4,CRUCE SEMAFORIZADO,LIMA,NaN,Constitucion,Comuna 1,NaN,NaN
4,-58.366325,-34.619239,2223.0,B-CR-02223,NaN,NaN,Huergo Ing. Av. y Calvo Carlos,NaN,Coordenada,Huergo- Carlos Calvo,4.0,TITILANTE A PEDIDO DE GCBA,Titilante por obra,Hasta la finalización de Obra de Paseo del Baj...,5,CRUCE SEMAFORIZADO,"HUERGO, ING. AV.",NaN,Puerto Madero,Comuna 1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,-58.382065,-34.591573,1182.0,B-CR-01182,NaN,NaN,9 de Julio Av. y Arroyo,NaN,Coordenada,NaN,4.0,NaN,NaN,NaN,212,NaN,9 DE JULIO AV.,NaN,Retiro,Comuna 1,NaN,NaN
3942,-58.454646,-34.650802,9654.0,B-CR-09654,NaN,NaN,Av. Castañares y Moreno Perito Av.,NaN,Coordenada,NaN,4.0,NaN,NaN,NaN,3682,CRUCE SEMAFORIZADO,CASTA\u00d1ARES AV.,NaN,Flores,Comuna 7,NaN,NaN
3943,-58.469395,-34.649631,9554.0,B-CR-09554,NaN,NaN,"Av. Perón Eva y Lacarra ""E""",NaN,Coordenada,NaN,4.0,NaN,NaN,NaN,3632,CRUCE SEMAFORIZADO,NaN,NaN,Parque Avellaneda,Comuna 9,NaN,NaN
3944,-58.469616,-34.649451,9555.0,B-CR-09555,NaN,NaN,Av. Perón Eva y Lacarra,NaN,Coordenada,NaN,4.0,NaN,NaN,NaN,3633,CRUCE SEMAFORIZADO,"PERON, EVA AV.",NaN,Parque Avellaneda,Comuna 9,NaN,NaN
